In [2]:
import vtk
import xml.etree.ElementTree as ET
import os
import numpy as np
from utilities import PrintProgress

def ReadVTPFile(FileName):
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(FileName)
    reader.Update()
    return reader

def CutPlane(Volume,Origin,Norm):
    plane=vtk.vtkPlane()
    plane.SetOrigin(Origin)
    plane.SetNormal(Norm)
    Slice=vtk.vtkCutter()
    Slice.GenerateTrianglesOff()
    Slice.SetCutFunction(plane)
    Slice.SetInputData(Volume)
    Slice.Update()
    return Slice.GetOutput()

def ReadVTIFile(FileName):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(FileName)
    reader.Update()
    return reader.GetOutput()

def WriteVTPFile(FileName,Data):
    writer=vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(FileName)
    writer.SetInputData(Data)
    writer.Update()

def SphereClip(volume_image,center,radius):
    sphere = vtk.vtkSphere()
    sphere.SetCenter(center)
    sphere.SetRadius(radius)

    clipper = vtk.vtkClipDataSet()
    clipper.SetInputData(volume_image)
    clipper.SetClipFunction(sphere)
    clipper.InsideOutOn()
    clipper.GetOutputInformation(1)
    clipper.Update()

    return clipper.GetOutput()

def PlaneClip(volume,center, a):
    box = vtk.vtkBox()
    box.SetBounds(
        center[0] - a, center[0] + a,
        center[1] - a, center[1] + a,
        center[2] - a, center[2] + a
    )

    clipper = vtk.vtkClipDataSet()
    clipper.SetInputData(volume)
    clipper.SetClipFunction(box)
    clipper.InsideOutOn()
    clipper.GetOutputInformation(1)
    clipper.Update()

    return clipper.GetOutput()

def gradient_filter(vtk_image):
    gradient_filter = vtk.vtkImageGradient()
    gradient_filter.SetInputData(vtk_image)
    gradient_filter.SetDimensionality(3)
    gradient_filter.Update()

    return gradient_filter.GetOutput()
    
def define_borders(gradient_image):
    magnitude_filter = vtk.vtkImageMagnitude()
    magnitude_filter.SetInputData(gradient_image)
    magnitude_filter.Update()
    
    return magnitude_filter.GetOutput()

def ExtractSurface(UnstrcturedGrid):
    geometry_filter = vtk.vtkGeometryFilter()
    geometry_filter.SetInputData(UnstrcturedGrid)
    geometry_filter.Update()

    return geometry_filter.GetOutput()

def WriteVTUFile(FileName,Data):
	writer=vtk.vtkXMLUnstructuredGridWriter()
	writer.SetFileName(FileName)
	writer.SetInputData(Data)
	writer.Update()

def clip_polydata_with_plane(polydata, origin, normal, inside_out=False):
    plane = vtk.vtkPlane()
    plane.SetOrigin(origin)
    plane.SetNormal(normal)

    clipper = vtk.vtkClipPolyData()
    clipper.SetInputData(polydata)
    clipper.SetClipFunction(plane)

    if inside_out:
        clipper.InsideOutOn()
    else:
        clipper.InsideOutOff()
    
    clipper.Update()

    return clipper.GetOutput()

In [119]:
InputFolder = "/Users/ana/Documents/AnahitaSeresti/04_Tesselation/KoenTesselation_SU03A/SimVascular_Edgard"
pathline_file = f"{InputFolder}/Paths/R_PL_1.pth"
ImagePath = f"{InputFolder}/Images/ct.vti"

In [120]:
OutputFolder = os.path.dirname(pathline_file)
VesselName = os.path.splitext(os.path.basename(pathline_file))[0]
OutputFolder = os.path.join(OutputFolder,"Segmentations")#VesselName)

In [104]:
os.system(f"mkdir {OutputFolder}")

mkdir: /Users/ana/Documents/AnahitaSeresti/04_Tesselation/KoenTesselation_SU03A/SimVascular_Edgard/Paths/Segmentations: File exists


256

In [121]:
with open(pathline_file, "r") as path:
    #path.readlines()
    tree = ET.parse(path)
root = tree.getroot()

direction_points = []
for direction_point in root.findall(".//path_point/tangent"):
    x = float(direction_point.attrib['x'])
    y = float(direction_point.attrib['y'])
    z = float(direction_point.attrib['z'])
    direction_points.append((x,y,z))

path_points = []
for path_point in root.findall(".//path_point/pos"):
    x = float(path_point.attrib['x'])
    y = float(path_point.attrib['y'])
    z = float(path_point.attrib['z'])
    path_points.append((x,y,z))

path_normals = []
for normal in root.findall(".//path_point/rotation"):
    x = float(normal.attrib['x'])
    y = float(normal.attrib['y'])
    z = float(normal.attrib['z'])
    path_normals.append((x,y,z))

NPoints = len(path_points)
print("the number of points in the centerline is:", NPoints)

binormal = []
for i in range(NPoints):
    binormal_ = np.cross(np.array(direction_points[i]),np.array(path_normals[i]))
    binormal_ /= np.linalg.norm(binormal_)
    binormal.append(binormal_)
    

the number of points in the centerline is: 105


In [111]:
Volume = ReadVTIFile(ImagePath)
Volume_Gradient = define_borders(gradient_filter(Volume))

In [76]:
def RotateVector(ray, normal, angle):
    angle = np.radians(angle)
    normal /= np.linalg.norm(normal)
    ray_rot = (ray * np.cos(angle) + np.cross(normal, ray) * np.sin(angle) + normal * np.dot(normal, ray) * (1 - np.cos(angle)))
    
    return ray_rot

def Line(point1, point2, res):
    line = vtk.vtkLineSource()
    line.SetPoint1(point1)
    line.SetPoint2(point2)
    line.SetResolution(res)
    line.Update()
    
    return line.GetOutput()

def ProbeFilter(InputData, SourceData):
    probe = vtk.vtkProbeFilter()
    probe.AddInputData(InputData)
    probe.SetSourceData(SourceData)
    probe.Update()
    
    return probe.GetOutput()


def EstimateVesselRadius2(CrossSectionGradient, CenterPoint, CL_direction):
    NRays_sample = 10
    ray_ = np.array([-CL_direction[2], 0, CL_direction[0]])
    res = 100
    angles = np.linspace(0, 360, NRays_sample, endpoint= False)
    Radius = 0
    counter = 0

    for angle in angles:
        ray_dir = RotateVector(ray_, CL_direction, angle)
        point2 = np.array([CenterPoint[0] + 100 * ray_dir[0], CenterPoint[1] + 100 * ray_dir[1], CenterPoint[2] + 100 * ray_dir[2]])
        ray  = Line(CenterPoint, point2, res)
        ray_projected = ProbeFilter(ray, CrossSectionGradient)

        sampled_data = ray_projected.GetPointData().GetScalars()
        gradients = np.array([sampled_data.GetTuple1(i) for i in range(sampled_data.GetNumberOfTuples())])

        boundary_idx = np.argmax(gradients)

        boundary_point = ray_projected.GetPoint(boundary_idx)
        radius = np.sqrt((boundary_point[0] - CenterPoint[0])**2 + (boundary_point[1] - CenterPoint[1])**2 + (boundary_point[2] - CenterPoint[2])**2)
        if radius > 4:
            radius = 0
            counter += 1
        Radius += radius
    if len(angles) - counter==0:
        Radius = 0
    else:
        Radius /= len(angles) - counter

    return Radius




In [122]:
def points_to_vtp(points):
    # Create VTK points
    vtk_points = vtk.vtkPoints()
    for point in points:
        vtk_points.InsertNextPoint(point)

    # Create a polyline
    polyline = vtk.vtkPolyLine()
    polyline.GetPointIds().SetNumberOfIds(len(points))
    for i in range(len(points)):
        polyline.GetPointIds().SetId(i, i)

    # Create a cell array to store the polyline
    cells = vtk.vtkCellArray()
    cells.InsertNextCell(polyline)

    # Create a polydata object
    polydata = vtk.vtkPolyData()
    polydata.SetPoints(vtk_points)
    polydata.SetLines(cells)

    return polydata

CenterLineVTP = points_to_vtp(path_points)


In [123]:
# Run this for small and short vessels only and skip the rest

def Tube(centerline):
    tube_filter = vtk.vtkTubeFilter()
    tube_filter.SetInputData(centerline)
    #tube_filter.SetRadius(Radius)  # Adjust this value to change thickness
    tube_filter.SetNumberOfSides(50)  # Higher = smoother tube
    #tube_filter.CappingOn()  # Close tube ends
    tube_filter.SetVaryRadiusToVaryRadiusByAbsoluteScalar()
    tube_filter.Update()
    return tube_filter.GetOutput()

r_max = 1.5
r_min = 0.6
Radius = np.arange(r_min, r_max, (r_max-r_min)/(NPoints))
print(NPoints, len(Radius))

radius_scalars = vtk.vtkDoubleArray()
radius_scalars.SetNumberOfComponents(1)
radius_scalars.SetName("Radius")

for i in range(NPoints):
    radius_scalars.InsertNextValue(Radius[NPoints - i -1])

CenterLine = CenterLineVTP
CenterLine.GetPointData().SetScalars(radius_scalars)

VesselSegmentation = Tube(CenterLine)

OutputFileName = f"{VesselName}_Segementation.vtp"
WriteVTPFile(os.path.join(OutputFolder,OutputFileName), VesselSegmentation)

105 105


In [78]:
from scipy.interpolate import interp1d

radius_scalars = vtk.vtkDoubleArray()
radius_scalars.SetNumberOfComponents(1)
radius_scalars.SetName("Radius")

a = 10
Radius = []
old_progress = 0
for i in range(NPoints):
    old_progress = PrintProgress(i, NPoints, old_progress)
    CrossSectionGrd = CutPlane(SphereClip(Volume_Gradient,path_points[i],2*a),path_points[i],direction_points[i])
    #radius = EstimateVesselRadius(CrossSectionGrd, path_points[i], path_normals[i]) #direction_points[i])
    radius = EstimateVesselRadius2(CrossSectionGrd, path_points[i], direction_points[i])
    Radius.append(radius)
    #radius_scalars.InsertNextValue(radius)



    Progress: 10%
    Progress: 20%
    Progress: 30%
    Progress: 40%
    Progress: 50%
    Progress: 60%
    Progress: 70%
    Progress: 80%
    Progress: 90%
    Progress: 100%


In [87]:
def CleanRadius(Radius, start_point = 4, end_point = 0.8):
    Radius = np.array(Radius)
    NPoints = len(Radius)

    expected_radii = np.linspace(start_point,end_point, NPoints)
    min_allowed = 0.25*expected_radii
    max_allowed = 1.1*expected_radii

    outliers = (Radius < min_allowed) | (Radius > max_allowed)

    valid_indices = np.where(~outliers)[0]
    valid_radii = Radius[valid_indices]

    interp_func = interp1d(valid_indices, valid_radii, kind='linear', fill_value='extrapolate')

    cleaned_radius = Radius.copy()
    cleaned_radius[outliers] = interp_func(np.where(outliers)[0])

    return cleaned_radius

CleanedRadius = CleanRadius(Radius)


In [88]:
def MovingAverageFilter(input_array, window_size = 5):
    return np.convolve(input_array, np.ones(window_size)/window_size, mode='same')

#SmoothedRadius = MovingAverageFilter(CleanedRadius,5)

from scipy.ndimage import gaussian_filter1d

SmoothedRadius = gaussian_filter1d(CleanedRadius, sigma = 2)


for i in range(NPoints):
    print(Radius[i], CleanedRadius[i], SmoothedRadius[i])
    radius_scalars.InsertNextValue(SmoothedRadius[i])

2.7878592076174282 2.7878592076174282 2.9310351018310365
0 3.0349861310353403 2.9088453191863977
3.2821130544532524 3.2821130544532524 2.8518564520183007
2.7808732440216377 2.7808732440216377 2.7630601263745236
2.3394450975904655 2.3394450975904655 2.6759224820352783
2.3342135445926844 2.3342135445926844 2.6320561564552514
2.663961187678937 2.663961187678937 2.6471258608229373
2.6801499080984152 2.6801499080984152 2.7020136189455557
2.899273684223438 2.899273684223438 2.7616360266602094
2.8713434293169686 2.8713434293169686 2.797611716152846
2.8925445642515113 2.8925445642515113 2.7989654440192364
2.8969357435941356 2.8969357435941356 2.7705584979413933
2.477634461234 2.477634461234 2.722908979204739
2.771392860610534 2.771392860610534 2.6607540213330254
2.8732355651794856 2.8732355651794856 2.5820120514144986
2.2600764600125682 2.2600764600125682 2.489411814603325
2.4808011888476353 2.4808011888476353 2.4001483643363923
2.2838660287081636 2.2838660287081636 2.3392202483092492
1.699957

In [89]:
CenterLine = CenterLineVTP
CenterLine.GetPointData().SetScalars(radius_scalars)

0

In [90]:
VesselSegmentation = Tube(CenterLine)

In [91]:
#CS_Grd_FileName = f"{VesselName}_CrossSection_{cs_loc}_Grd.vtp"
#WriteVTPFile(os.path.join(OutputFolder,CS_Grd_FileName), CrossSectionGrd)

OutputFileName = f"{VesselName}_Segementation.vtp"
WriteVTPFile(os.path.join(OutputFolder,OutputFileName), VesselSegmentation)